<a href="https://colab.research.google.com/github/sbbaik/small_bert/blob/main/Emotion_Classifier_Fine_tuning_for_Smart_Bulb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import os
import math
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from tokenizers import BertWordPieceTokenizer
from datasets import load_dataset
from transformers import AdamW # AdamW 옵티마이저 사용 (BERT 학습에 권장)

# --- 1. Colab 환경 설정 및 구글 드라이브 마운트 ---
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/small_bert_korean'

# 디렉토리가 없으면 생성 (사전 학습 시 생성되었겠지만, 안전을 위해 다시 확인)
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    print(f"디렉토리 '{SAVE_PATH}'가 생성되었습니다.")

# --- 2. 하이퍼파라미터 정의 (사전 학습 시와 동일) ---
d_model = 768
n_head = 12
num_layers = 6
dim_feedforward = d_model * 4
max_len = 50
dropout = 0.1
num_emotion_classes = 2 # NSMC는 긍정(1), 부정(0) 2가지 클래스

# --- 3. Positional Encoding 클래스 정의 (사전 학습 시와 동일) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=50):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# --- 4. SmallBERT 모델 클래스 정의 (사전 학습 시와 동일) ---
# 이 모델은 사전 학습된 가중치를 로드할 백본이 됩니다.
class SmallBERT(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, num_layers, dim_feedforward, max_len, dropout):
        super(SmallBERT, self).__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_head,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # 사전 학습 시 사용했던 output_layer는 MLM용이므로, 여기서는 사용하지 않지만
        # 모델 구조를 동일하게 유지하기 위해 일단 정의해 둡니다.
        # 실제 분류 모델에서는 이 레이어를 사용하지 않거나, 다른 분류 헤드로 대체됩니다.
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        x = self.embedding(src) * math.sqrt(d_model)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x)
        return output # MLM output_layer를 통과시키지 않고 인코더의 최종 출력만 반환

# --- 5. 감정 분류를 위한 모델 클래스 정의 ---
class EmotionClassifier(nn.Module):
    def __init__(self, small_bert_model, num_classes):
        super(EmotionClassifier, self).__init__()
        self.bert = small_bert_model # 사전 학습된 SmallBERT 모델을 백본으로 사용
        # 분류 헤드 추가: SmallBERT의 출력 (d_model)을 num_classes로 변환
        self.classifier = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout) # 분류 헤드에 드롭아웃 적용

    def forward(self, input_ids):
        # SmallBERT의 forward를 호출하여 인코더의 최종 출력을 얻음
        # BERT는 보통 [CLS] 토큰의 출력을 분류에 사용하므로, 첫 번째 토큰의 출력을 가져옵니다.
        pooled_output = self.bert(input_ids)[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# --- 6. 토크나이저 로드 (사전 학습된 토크나이저 사용) ---
# 사전 학습 시 생성된 vocab.txt 파일이 존재해야 합니다.
tokenizer = BertWordPieceTokenizer.from_file(
    os.path.join(SAVE_PATH, 'vocab.txt'),
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False
)
vocab_size = tokenizer.get_vocab_size()

# --- 7. NSMC 데이터셋 로드 및 전처리 ---
print("네이버 영화 리뷰 감성분석 데이터셋(NSMC)을 로드합니다...")
nsmc_dataset_train = load_dataset('nsmc', split='train')
nsmc_dataset_test = load_dataset('nsmc', split='test')
print(f"NSMC 학습 데이터셋 로드 완료. 총 {len(nsmc_dataset_train)}개의 샘플.")
print(f"NSMC 테스트 데이터셋 로드 완료. 총 {len(nsmc_dataset_test)}개의 샘플.")

class EmotionDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.texts = []
        self.labels = []

        for entry in hf_dataset:
            text = entry['document']
            label = entry['label']
            if text is None: # None 값 처리
                continue
            self.texts.append(text)
            self.labels.append(label)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # 토큰화 및 패딩/트렁케이션
        encoding = self.tokenizer.encode(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True
        )

        input_ids = torch.tensor(encoding.ids, dtype=torch.long)
        # BERT 모델은 attention_mask를 사용하여 패딩 토큰을 무시합니다.
        attention_mask = torch.tensor(encoding.attention_mask, dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, # BERT 모델은 attention_mask를 사용하지만, SmallBERT는 현재 input_ids만 받으므로 여기서는 사용하지 않습니다.
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 인스턴스 생성
train_dataset = EmotionDataset(nsmc_dataset_train, tokenizer, max_len)
test_dataset = EmotionDataset(nsmc_dataset_test, tokenizer, max_len)

# 데이터로더 설정
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- 8. 모델 인스턴스 생성 및 파인튜닝 루프 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 사전 학습된 SmallBERT 모델 로드
# output_layer는 MLM용이므로, load_state_dict 시 strict=False를 사용하여 무시할 수 있습니다.
# 또는 SmallBERT 클래스에서 output_layer를 제거하고 로드할 수도 있습니다.
# 여기서는 모델 구조를 변경하지 않고 로드 후 EmotionClassifier에 전달합니다.
pretrained_bert_model = SmallBERT(vocab_size, d_model, n_head, num_layers, dim_feedforward, max_len, dropout).to(device)
# 사전 학습된 가중치 로드 (MLM 헤드를 제외하고 로드)
# strict=False를 사용하면 모델의 output_layer와 같이 로드하려는 state_dict에 없는 키를 무시합니다.
pretrained_bert_model.load_state_dict(torch.load(os.path.join(SAVE_PATH, 'final_small_bert.pt'), map_location=device), strict=False)


# EmotionClassifier 모델 생성
emotion_model = EmotionClassifier(pretrained_bert_model, num_emotion_classes).to(device)

# 옵티마이저 및 손실 함수
# AdamW는 BERT 파인튜닝에 널리 사용됩니다.
optimizer = AdamW(emotion_model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss() # 분류 문제이므로 CrossEntropyLoss 사용

# 파인튜닝 루프
num_finetune_epochs = 3 # 파인튜닝은 보통 적은 에포크로 진행
print("\n감정 분류 모델 파인튜닝을 시작합니다...")
for epoch in range(num_finetune_epochs):
    emotion_model.train()
    total_train_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device) # 감정 레이블

        optimizer.zero_grad()
        logits = emotion_model(input_ids) # 모델 출력은 로짓

        loss = criterion(logits, labels) # 로짓과 레이블로 손실 계산
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Finetune Epoch [{epoch+1}/{num_finetune_epochs}], Train Loss: {avg_train_loss:.4f}")

    # (선택 사항) 테스트 데이터셋으로 성능 평가
    emotion_model.eval()
    total_eval_accuracy = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            logits = emotion_model(input_ids)
            predictions = torch.argmax(logits, dim=-1)
            total_eval_accuracy += (predictions == labels).sum().item()

    avg_eval_accuracy = total_eval_accuracy / len(test_dataset)
    print(f"Finetune Epoch [{epoch+1}/{num_finetune_epochs}], Test Accuracy: {avg_eval_accuracy:.4f}")

# --- 9. 최종 감정 분류 모델 저장 ---
final_emotion_model_path = os.path.join(SAVE_PATH, 'final_emotion_classifier.pt')
torch.save(emotion_model.state_dict(), final_emotion_model_path)
print(f"최종 감정 분류 모델이 {final_emotion_model_path}에 저장되었습니다.")

# --- 10. 감정 예측 및 전구 파라미터 변환 함수 ---
# 감정 레이블 매핑 (NSMC: 0=부정, 1=긍정)
emotion_labels = {0: "부정적", 1: "긍정적"}

# 전구 파라미터 정의 (예시 값)
# 색상 (RGB 또는 Hex), 밝기 (0-100), 색온도 (Kelvin)
# 실제 전구 제어 API에 맞게 조정해야 합니다.
bulb_parameters = {
    "부정적": {"color": "#ADD8E6", "brightness": 30, "color_temp": 6500}, # 연한 파랑, 어둡고 차가움
    "긍정적": {"color": "#FFFF00", "brightness": 80, "color_temp": 2700}, # 노랑, 밝고 따뜻함
    # 더 많은 감정 클래스가 있다면 여기에 추가
    # 예: "슬픔": {"color": "#4682B4", "brightness": 20, "color_temp": 7000},
    # 예: "기쁨": {"color": "#FFD700", "brightness": 90, "color_temp": 2500},
    # 예: "평온": {"color": "#90EE90", "brightness": 60, "color_temp": 4000},
    # 예: "분노": {"color": "#FF4500", "brightness": 70, "color_temp": 3500},
}

def analyze_and_control_bulb(text, model, tokenizer, device, emotion_map, bulb_map):
    model.eval() # 평가 모드
    encoding = tokenizer.encode(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True
    )
    input_ids = torch.tensor([encoding.ids], dtype=torch.long).to(device)

    with torch.no_grad():
        logits = model(input_ids)
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = torch.argmax(probabilities, dim=-1).item()

    predicted_emotion = emotion_map[predicted_class_id]
    bulb_params = bulb_map.get(predicted_emotion, {"color": "#FFFFFF", "brightness": 50, "color_temp": 4000}) # 기본값

    print(f"문장: '{text}'")
    print(f"예측된 감정: {predicted_emotion} (확률: {probabilities[0, predicted_class_id].item():.4f})")
    print(f"스마트 전구 파라미터: 색상={bulb_params['color']}, 밝기={bulb_params['brightness']}, 색온도={bulb_params['color_temp']}")
    print("-" * 50)
    return predicted_emotion, bulb_params

# --- 11. 감정 분석 및 전구 제어 예시 실행 ---
print("\n감정 분석 및 전구 제어 예시:")
analyze_and_control_bulb("너무 감사하다", emotion_model, tokenizer, device, emotion_labels, bulb_parameters)
analyze_and_control_bulb("기분이 꿀꿀하네...", emotion_model, tokenizer, device, emotion_labels, bulb_parameters)
analyze_and_control_bulb("오늘 날씨가 정말 좋다!", emotion_model, tokenizer, device, emotion_labels, bulb_parameters)
analyze_and_control_bulb("아, 정말 짜증나!", emotion_model, tokenizer, device, emotion_labels, bulb_parameters)
analyze_and_control_bulb("이 영화는 정말 재미없다.", emotion_model, tokenizer, device, emotion_labels, bulb_parameters)